### rag

In [ ]:
# !pip install -qU "langchain[openai]"
# !pip install -U langchain-cli
# !pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# !pip install -U langchain-openai
# !pip install dotenv
# !pip install -qU langchain-qdrant
# !pip install bs4

In [64]:
import os, dotenv
dotenv.load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [15]:
import sys ,os

sys.path.append(os.path.abspath(os.path.join('..')))

In [17]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")



In [63]:
model.invoke("qual o produto mais vendido ? ")

AIMessage(content='O produto mais vendido pode variar conforme o contexto, o local e o período analisado. Em geral, produtos como alimentos básicos, eletrônicos, roupas e itens de higiene pessoal costumam estar entre os mais vendidos. Por exemplo:\n\n1. **Alimentos**: Arroz, feijão, e itens de café da manhã como pães e cereais.\n2. **Eletrônicos**: Smartphones e acessórios como fones de ouvido e carregadores.\n3. **Roupas**: Moda básica, como camisetas e calças.\n\nSe você estiver se referindo a um setor específico ou a uma região, por favor, forneça mais detalhes para que eu possa oferecer uma resposta mais precisa!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 14, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-

In [19]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [20]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [1]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

NameError: name 'embeddings' is not defined

In [22]:
# from langchain_qdrant import QdrantVectorStore
# from qdrant_client import QdrantClient

# client = QdrantClient(":5fe6e0b9-3c61-4ede-8f55-b3ab7acf7eaf:")
# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="nexmuv",
#     embedding=embeddings,
# )

In [ ]:
#Web scraping loader
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)


In [34]:
# !pip install -U langchain-community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
loader = PyPDFLoader(
                file_path = r"C:\Users\OneTi2\Desktop\health_check\rag_chat\src\George-Orwell-1984.pdf",
                # headers = None
                # password = None,
                mode = "single",
                pages_delimiter = "\n\f",
                # extract_images = True,
                # images_parser = RapidOCRBlobParser(),
            )
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)



In [41]:
os.getcwd()

'c:\\Users\\OneTi2\\Desktop\\Projetos\\teste\\rag_chat_ort\\test'

In [55]:
from langchain_community.document_loaders import CSVLoader

# loader excel
loader = document_loaders.CSVLoader(
    file_path = r"C:\Users\OneTi2\Desktop\Projetos\teste\rag_chat_ort\src\relatorio_geral_hortifruti.csv",

)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)


In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts.chat import ChatPromptTemplate



_ = vector_store.add_documents(documents=all_splits)


prompt = ChatPromptTemplate.from_template(
    """
    Você é um assistente para tarefas de perguntas e respostas. 
    Use os seguintes trechos de contexto recuperados para responder à pergunta. 
    Se você não souber a resposta, apenas diga que não sabe.
    Use no máximo três frases e mantenha a resposta concisa.
    Seja amigavel e educado.
    \nPergunta: {question} \nContexto: {context} \nResposta:"""
)


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [62]:
response = graph.invoke({"question": "Qual o produto mais vendido?"})
print(response["answer"])

O produto mais vendido é a "Broa Com Gotas De Chocolate". Ela está na categoria de confeitaria e teve várias perdas registradas, mas é mencionada repetidamente no contexto. Se precisar de mais informações, é só perguntar!


In [ ]:
import sqlite3

conn = sqlite3.connect(r'C:\Users\OneTi2\Desktop\health_check\health_check.db')
cursor = conn.cursor()

# Criando o esquema do banco
cursor.execute('''
    SELECT * FROM sales as sl
    JOIN customers as ct ON ct.id == sl.customer_id
    JOIN products as pdt ON pdt.id == sl.product_id
''' 
)
data = cursor.fetchall()



OperationalError: unable to open database file

In [46]:
import csv

csv_file_path = r'C:\Users\OneTi2\Desktop\Projetos\teste\rag_chat_ort\src\relatorio_geral_hortifruti.csv'

column_names = [description[0] for description in cursor.description]

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(column_names) 
    writer.writerows(data)  

print(f'Dados exportados para {csv_file_path}')

NameError: name 'cursor' is not defined

In [21]:

import requests
print(requests.get('http://176.52.134.170:8080/?option=Dynamcs - Download Faturas').text)

ConnectTimeout: HTTPConnectionPool(host='176.52.134.170', port=8080): Max retries exceeded with url: /?option=Dynamcs%20-%20Download%20Faturas (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000022B56DA2600>, 'Connection to 176.52.134.170 timed out. (connect timeout=None)'))

In [79]:
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\OneTi2\Desktop\Projetos\teste\rag_chat_ort\src\relatorio_geral_hortifruti.csv')
df.head(2)


,Unnamed: 0,Data,QuantidadePerdida,SetorProduto,CategoriaProduto,NomeProdutoCompleto,PrecoVenda,Fornecedor,PrecoCusto,TipoPerda,Produto,AcaoReducaoPerda,CustoReducaoPerda,Loja,Ano,NomeMes,NomeDia,ValorTotalPerdaVenda,ValorTotalPerdaCusto
0,0,2024-01-10,15,Hortifruti,Verduras,Acelga Kg,6.99,Terra Boa,4.0,Deterioração rápida,Acelga,Sistema de Hidroresfriamento,500,Itaim Bibi,2024,janeiro,quarta-feira,104.85,60.0
1,1,2024-11-26,30,Hortifruti,Verduras,Acelga Kg,6.99,Terra Boa,4.0,Deterioração rápida,Acelga,Sistema de Hidroresfriamento,500,Shop. Market Place,2024,novembro,terça-feira,209.70,120.0


In [45]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = CSVLoader(file_path=csv_file_path,
                csv_args={
                'delimiter': ',',
                'quotechar': '"',
                'fieldnames': df.columns
            })

docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)


NameError: name 'csv_file_path' is not defined

In [14]:
all_splits[0]

Document(metadata={'source': 'C:\\Users\\OneTi2\\Desktop\\health_check\\rag_chat\\src\\sales.csv', 'row': 0}, page_content='id: id\nproduct_id: product_id\ncustomer_id: customer_id\nquantity: quantity\ntotal_amount: total_amount\nsale_date: sale_date\nid.1: id\nname: name\nemail: email\ncreated_at: created_at\nid.2: id\nsku: sku\nname.1: name\ncategory: category\nprice: price')

In [15]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts.chat import ChatPromptTemplate



# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# prompt = hub.pull("rlm/rag-prompt")
prompt = ChatPromptTemplate.from_template(
    """
    Você é um assistente para tarefas de perguntas e respostas. 
    Use os seguintes trechos de contexto recuperados para responder à pergunta. 
    Se você não souber a resposta, apenas diga que não sabe.
    Use no máximo três frases e mantenha a resposta concisa.
    Seja amigavel e educado.
    \nPergunta: {question} \nContexto: {context} \nResposta:"""
)


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder2 = StateGraph(State).add_sequence([retrieve, generate])
graph_builder2.add_edge(START, "retrieve")
graph = graph_builder2.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
response = graph.invoke({"question": "Qual o produto mais vendido ?"})
print(response["answer"])

O produto mais vendido, com base nas informações disponíveis, parece ser o Product E, com um total de 8 unidades vendidas. Se precisar de mais informações, estou à disposição para ajudar!


### ort

In [8]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [6]:
import pandas as pd

In [31]:
os.getcwd()

'c:\\Users\\OneTi2\\Desktop\\Projetos\\teste\\rag_chat_ort\\test'

In [2]:
import requests


In [2]:
import requests

response = requests.get('http://127.0.0.1:8000/?question=qual o produto mais caro')
print(response)

<Response [500]>


In [ ]:
df = pd.read_excel(r'../src/Relatório Gerencial - Hortifruti.xlsx').to_csv('relatorio_geral_hortifruti.csv')


In [11]:
df.columns

Index(['Data', 'QuantidadePerdida', 'SetorProduto', 'CategoriaProduto',
       'NomeProdutoCompleto', 'PrecoVenda', 'Fornecedor', 'PrecoCusto',
       'TipoPerda', 'Produto', 'AcaoReducaoPerda', 'CustoReducaoPerda', 'Loja',
       'Ano', 'NomeMes', 'NomeDia', 'ValorTotalPerdaVenda',
       'ValorTotalPerdaCusto'],
      dtype='object')